In [1]:
import numpy as np
from functools import reduce

In [2]:
data = 10*np.random.random(size=(50,3)).astype(np.float64)

In [3]:
data[0:10]

array([[ 8.52072465,  4.79744237,  9.96234095],
       [ 3.7235221 ,  1.11097119,  8.32738923],
       [ 0.80638772,  7.59114942,  2.72489623],
       [ 5.81189919,  7.08426853,  3.79595342],
       [ 1.0519065 ,  6.35617466,  0.57257328],
       [ 6.42945697,  9.48415513,  6.24193657],
       [ 3.22458364,  7.05208869,  1.44799552],
       [ 9.38381633,  2.99118814,  1.00208586],
       [ 6.69232493,  7.76201712,  4.85847647],
       [ 3.60508325,  6.59751882,  7.08843401]])

In [4]:
def sample_data_from_eqs(data, n_samples=100):
    for coeffs in data:
        eq = polynomial(coeffs)
        yield [(x, eq(x)) for x in np.linspace(0, 10, n_samples)]

In [5]:
def polynomial(coeffs):
    return lambda x: sum([a*x**n for n, a in enumerate(coeffs)])

In [6]:
def rmse(dataset, eq):
    total = 0
    for sample in dataset:
        val = eq(sample[0])
        total += (val - sample[1]) ** 2
    return (total / len(dataset)) ** (1/2)

In [7]:
def err(point, eq):
    return -point[1] + eq(point[0])
def dCda(dataset, eq):
    num = 0
    den = 0
    n = float(len(dataset))
    for point in dataset:
        num += 2 * point[0] ** 2 * err(point, eq)
        den += np.power(err(point, eq), 2)
    return num / (2 * n * (den / n) ** (1/2))
def dCdb(dataset, eq):
    num = 0
    den = 0
    n = float(len(dataset))
    for point in dataset:
        num += 2 * point[0] * err(point, eq)
        den += np.power(err(point, eq), 2)
    return num / (2 * n * (den / n) ** (1/2))
def dCdc(dataset, eq):
    den = 0
    n = float(len(dataset))
    for point in dataset:
        den += np.power(err(point, eq), 2)
    return 1 / (2 * n * (den / n) ** (1/2))
def d_dc(dataset, eq, p):
    num = 0
    den = 0
    n = float(len(dataset))
    for point in dataset:
        num += 2 * point[0] ** p * err(point, eq)
        den += np.power(err(point, eq), 2)
    return num / (2 * n * (den / n) ** (1/2))

In [31]:
lr = 0.1
costs1 = []
costs2 = []
costs3 = []
break_costs = []
_data = np.empty_like(data)
epochs = 1
for _ in range(epochs):
    print('epoch: ' + str(_) + '\r')
    for sample_num, sample in enumerate(sample_data_from_eqs(data, 10)):
        coeffs = 10*np.random.normal(0,1,3).astype(np.float_)
        last_cost = None
        print("new sample ... ", sample_num, " of ", len(data))
        for i in range(600):
            learning_rate = lr / (i + 1) + 0.00001
            eq = polynomial(coeffs)
            cost = rmse(sample, eq)
            if not last_cost:
                last_cost = cost
            elif cost > last_cost:
                break_costs.append(cost)
                # break
            last_cost = cost
            coeffs -= [learning_rate*d_dc(sample, eq, n) for n, a in enumerate(coeffs)]
            if i == 100:
                costs3.append(cost)
            if i == 200:
                costs2.append(cost)
        else:
            eq = polynomial(coeffs)
            costs1.append(rmse(sample,
                               eq))
        _data[sample_num] = coeffs
costs1 = np.array([val for val in costs1 if not np.isnan(val)])
costs2 = np.array([val for val in costs2 if not np.isnan(val)])
costs3 = np.array([val for val in costs3 if not np.isnan(val)])
break_costs = np.array([val for val in break_costs if not np.isnan(val)])
print('300', costs1.mean())
print('200', costs2.mean())
print('100', costs3.mean())
print('breaking', len(break_costs), break_costs.mean())

epoch: 0
new sample ...  0  of  50
new sample ...  1  of  50
new sample ...  2  of  50
new sample ...  3  of  50
new sample ...  4  of  50
new sample ...  5  of  50
new sample ...  6  of  50
new sample ...  7  of  50
new sample ...  8  of  50
new sample ...  9  of  50
new sample ...  10  of  50
new sample ...  11  of  50
new sample ...  12  of  50
new sample ...  13  of  50
new sample ...  14  of  50
new sample ...  15  of  50
new sample ...  16  of  50
new sample ...  17  of  50
new sample ...  18  of  50
new sample ...  19  of  50
new sample ...  20  of  50
new sample ...  21  of  50
new sample ...  22  of  50
new sample ...  23  of  50
new sample ...  24  of  50
new sample ...  25  of  50
new sample ...  26  of  50
new sample ...  27  of  50
new sample ...  28  of  50
new sample ...  29  of  50
new sample ...  30  of  50
new sample ...  31  of  50
new sample ...  32  of  50
new sample ...  33  of  50
new sample ...  34  of  50
new sample ...  35  of  50
new sample ...  36  of  50
ne

In [30]:
np.random.normal(0,1,3)

array([ 0.32090988, -0.73432891, -0.81031086])